In [ ]:
# ================================================
# 1. Install dependencies (run once if needed)
# ================================================
# !pip install transformers datasets scikit-learn pandas torch

In [1]:
# ================================================
# 2. Import libraries
# ================================================
import pandas as pd
import numpy as np
import torch
import random
import re
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback,
    TrainerCallback
)
from datasets import Dataset

In [2]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [4]:
# ================================================
# 3. Load the data
# ================================================
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

print(train_df.head())
print(test_df.head())

# ================================================
# 4. Define light preprocessing (safe for GPT-2)
# ================================================
def clean_text_gpt2(text):
    text = str(text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

train_df['text'] = train_df['text'].apply(clean_text_gpt2)
test_df['text'] = test_df['text'].apply(clean_text_gpt2)

# ================================================
# 5. Convert labels to text prompts
# ================================================
label_map = {0: 'negative', 1: 'neutral', 2: 'positive'}

# Prepare prompt and target
train_df['input_text'] = "Tweet: " + train_df['text'] + " Sentiment:"
train_df['target_text'] = train_df['label'].map(label_map)

test_df['input_text'] = "Tweet: " + test_df['text'] + " Sentiment:"

# ================================================
# 6. Train-validation split
# ================================================
train_data, val_data = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=42)

# ================================================
# 7. Load tokenizer
# ================================================
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 workaround for padding

# ================================================
# 8. Tokenization function
# ================================================
def tokenize_function(examples):
    inputs = tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target_text'], padding="max_length", truncation=True, max_length=10)
    inputs["labels"] = labels["input_ids"]
    return inputs

# Apply to train and val
train_dataset = Dataset.from_pandas(train_data[['input_text','target_text']])
val_dataset = Dataset.from_pandas(val_data[['input_text','target_text']])

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# ================================================
# 9. Load model (GPT-2 Causal LM)
# ================================================
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

# ================================================
# 10. Define Data Collator for LM
# ================================================
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

                                                text  label
0  $BYND - JPMorgan reels in expectations on Beyo...      0
1  $CCL $RCL - Nomura points to bookings weakness...      0
2  $CX - Cemex cut at Credit Suisse, J.P. Morgan ...      0
3  $ESS: BTIG Research cuts to Neutral https://t....      0
4  $FNKO - Funko slides after Piper Jaffray PT cu...      0
   id                                               text
0   0  ETF assets to surge tenfold in 10 years to $50...
1   1  Here’s What Hedge Funds Think Evolution Petrol...
2   2  $PVH - Phillips-Van Heusen Q3 2020 Earnings Pr...
3   3  China is in the process of waiving retaliatory...
4   4  Highlight: “When growth is scarce, investors s...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

Map:   0%|          | 0/7634 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1909 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
# ================================================
# 11. Define Trainer and Training Arguments
# ================================================
training_args = TrainingArguments(
    output_dir="./results_gpt2_causal",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.05,
    warmup_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_total_limit=1,
    logging_dir="./logs_gpt2_causal",
    logging_steps=10,
    report_to="none"
)

# Simple logger callback (optional, tracks losses)
class LossLogger(TrainerCallback):
    def __init__(self):
        self.eval_losses = []
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        self.eval_losses.append(metrics["eval_loss"])

logger = LossLogger()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[logger, EarlyStoppingCallback(early_stopping_patience=2)]
)

# ================================================
# 12. Train model
# ================================================
trainer.train()

<ipython-input-5-2348231881>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.101900,3.373321
2,2.762200,3.305345
3,2.324100,3.333640
4,2.210400,3.370019


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=3820, training_loss=2.820477886599396, metrics={'train_runtime': 1263.0915, 'train_samples_per_second': 60.439, 'train_steps_per_second': 7.561, 'total_flos': 1994703372288000.0, 'train_loss': 2.820477886599396, 'epoch': 4.0})

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:
# ================================================
# 13. Inference on test set
# ================================================
# Prepare test set for generation:
test_inputs = test_df['input_text'].tolist()
test_encodings = tokenizer(test_inputs, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Generate predictions
generated_ids = model.generate(
    input_ids=test_encodings['input_ids'],
    attention_mask=test_encodings['attention_mask'],
    max_new_tokens=5
)

# Decode predictions
predictions = tokenizer.batch_decode(generated_ids[:, test_encodings['input_ids'].shape[1]:], skip_special_tokens=True)

# Map back to numeric labels
reverse_label_map = {'negative': 0, 'neutral': 1, 'positive': 2}

# Apply simple cleaning to predictions
predictions_clean = [re.sub(r'\W+', '', pred.strip().split()[0].lower()) for pred in predictions]
pred_labels = [reverse_label_map.get(pred, 1) for pred in predictions_clean]  # default to neutral if unknown


# ================================================
# 14. Save submission file
# ================================================
test_df['label'] = pred_labels
test_df[['id','label']].to_csv("pred_decoder_causal.csv", index=False)
print("Prediction file saved as pred_decoder_causal.csv")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2479: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)